# Examples of how to run INEGIpy to get data from INEGI's API

@roman_avj

18 mar 24

In [35]:
import os
import pandas as pd
import yaml
from dotenv import load_dotenv

In [2]:
# load environment variables
load_dotenv()

True

In [71]:
# get config file
with open('../config.yaml', 'r') as file:
    config = yaml.safe_load(file)
config

{'inegi': {'series': {'inpc': 628194},
  'starting_date': datetime.date(2018, 1, 1)},
 'banxico': {'series': {'dollar_fx': 'SF63528', 'tiie': 'SF43783'},
  'starting_date': datetime.date(2018, 1, 1)}}

# INEGI API

In [72]:
from INEGIpy import Indicadores
from datetime import datetime

In [73]:
# inegi class
inegi_api = Indicadores(token=os.getenv('API_INEGI'))

In [74]:
?inegi_api.obtener_df

Signature:
inegi_api.obtener_df(
    indicadores: 'str|list',
    nombres: 'str|list' = None,
    clave_area: str = '00',
    inicio: str = None,
    fin: str = None,
    banco: str = None,
    metadatos: bool = False,
)
Docstring:
Regresa un DataFrame con la información de los indicadores proporcionada por el API del INEGI. Si metadatos = True regresa un segundo DataFrame con las claves de los metadatos del indicador. 

Parametros
-----------
indicadores: str/list. Clave(s) de los indicadores a consultar.
nombres: list/str, opcional. Nombre(s) de las columas del DataFrame. De no proporcionarse, se usarán los indicadores.
clave_area: str. Clave de dos a cinco caracteres que indentifica el área geográfica de acuerdo con el Marco Geoestadístico. Para definir el total nacional se especifica '00'. Este campo solo aplica para los indicadores del Bando de Indicadores (BISE), no aplica para los del Banco de Información Económica (BIE).
                            Dos dígitos para incluir nive

In [76]:
# get inpc
df_inpc = inegi_api.obtener_df(
    indicadores = '628194',
    nombres = 'inpc',
    inicio=config['inegi']['starting_date'].strftime('%Y-%m-%d'),
    )
# change the name of the index
df_inpc.index.name = 'date'
df_inpc

/Users/ravj/opt/anaconda3/envs/arquitectura/lib/python3.11/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)


,inpc
date,
2018-01-01,98.795000
2018-02-01,99.171374
2018-03-01,99.492157
2018-04-01,99.154847
2018-05-01,98.994080
...,...
2023-10-01,130.609000
2023-11-01,131.445000
2023-12-01,132.373000


# Banxico

In [6]:
from sie_banxico import SIEBanxico

In [42]:
# get data from Banxico
banxico_api = SIEBanxico(
    token=os.getenv('API_BANXICO'),
    id_series=list(config['banxico']['series'].values()),
    language='en'
)

In [52]:
# get FX rate and TIIE
today = datetime.today().strftime('%Y-%m-%d')
series_data = banxico_data.get_timeseries_range(
    init_date=config['banxico']['starting_date'].strftime('%Y-%m-%d'),
    end_date=today
)
series_data

{'bmx': {'series': [{'idSerie': 'SF63528',
    'titulo': 'Exchange rate historical series U.S. dollar - MXN exchange rate since 1954',
    'datos': [{'fecha': '02/01/2018', 'dato': '19.48990'},
     {'fecha': '03/01/2018', 'dato': '19.37170'},
     {'fecha': '04/01/2018', 'dato': '19.24270'},
     {'fecha': '05/01/2018', 'dato': '19.27370'},
     {'fecha': '08/01/2018', 'dato': '19.23230'},
     {'fecha': '09/01/2018', 'dato': '19.30650'},
     {'fecha': '10/01/2018', 'dato': '19.24050'},
     {'fecha': '11/01/2018', 'dato': '19.34330'},
     {'fecha': '12/01/2018', 'dato': '19.04460'},
     {'fecha': '15/01/2018', 'dato': '18.81130'},
     {'fecha': '16/01/2018', 'dato': '18.83620'},
     {'fecha': '17/01/2018', 'dato': '18.65500'},
     {'fecha': '18/01/2018', 'dato': '18.63610'},
     {'fecha': '19/01/2018', 'dato': '18.58130'},
     {'fecha': '22/01/2018', 'dato': '18.71180'},
     {'fecha': '23/01/2018', 'dato': '18.79800'},
     {'fecha': '24/01/2018', 'dato': '18.51780'},
     {

In [85]:
dict_df = {}
for serie in series_data['bmx']['series']:
    df = pd.DataFrame(serie['datos'])
    df['date'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
    df.set_index('date', inplace=True)
    df.drop(columns=['fecha'], inplace=True)
    df.index = df.index.strftime('%Y-%m-%d')
    df = df.rename(columns={'dato': serie['idSerie']})
    dict_df[serie['idSerie']] = df

# join dataframes by index
df = pd.concat(dict_df.values(), axis=1)

# index to DateTimeIndex
df.index = pd.to_datetime(df.index)

In [86]:
df

,SF63528,SF43783
date,,
2018-01-02,19.48990,7.6311
2018-01-03,19.37170,7.6275
2018-01-04,19.24270,7.6250
2018-01-05,19.27370,7.6250
2018-01-08,19.23230,7.6292
...,...,...
2024-03-11,16.80830,11.4825
2024-03-12,16.82720,11.4724
2024-03-13,16.71270,11.4748


In [87]:
df.index

DatetimeIndex(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
               '2018-01-08', '2018-01-09', '2018-01-10', '2018-01-11',
               '2018-01-12', '2018-01-15',
               ...
               '2024-03-04', '2024-03-05', '2024-03-06', '2024-03-07',
               '2024-03-08', '2024-03-11', '2024-03-12', '2024-03-13',
               '2024-03-14', '2024-03-15'],
              dtype='datetime64[ns]', name='date', length=1563, freq=None)

In [88]:
df_inpc.index

DatetimeIndex(['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
               '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
               '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
               '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
               '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01',
               '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',
               '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01',
               '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01',
               '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
               '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01',
               '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01',
      

In [90]:
# join df with inpc
df2 = df.join(df_inpc, how='left')

# ffll inpc
df2['inpc'] = df2['inpc'].ffill()

df2.tail(35)

,SF63528,SF43783,inpc
date,,,
2024-01-26,17.16570,11.4995,132.373
2024-01-29,17.23330,11.4985,132.373
2024-01-30,17.19320,11.4990,132.373
2024-01-31,17.16330,11.5000,132.373
2024-02-01,17.13350,11.5012,133.681
2024-02-02,17.14470,11.4993,133.681
2024-02-06,17.03570,11.4984,133.681
2024-02-07,17.03980,11.4980,133.681
2024-02-08,17.10230,11.4990,133.681
